In [ ]:
# savepoint 저장 환경 설정
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='/content/drive/MyDrive/project_olist_ecommerce/checkpoints',

    # 평가 및 저장 전략 (500 스텝 주기)
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=200,
    save_steps=200,
    logging_steps=200,

    # 모델 관리 (용량 절약 및 최적 모델 로드)
    save_total_limit=2,
    load_best_model_at_end=True,

    fp16=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2
)

In [ ]:
!pip install -U bitsandbytes accelerate

In [ ]:
# 라이브러리
from google.colab import drive
import pandas as pd

# 데이터셋 로드
drive.mount('/content/drive')
dataset = '/content/drive/MyDrive/브라질 이커머스/project_olist_ecommerce/project_dataset/olist_reviews_clean.csv'
df_NLP = pd.read_csv(dataset)

print(df_NLP.info())
print(df_NLP.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33491 entries, 0 to 33490
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                33491 non-null  object 
 1   review_comment_message  33491 non-null  object 
 2   product_category_name   33491 non-null  object 
 3   labels                  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 1.0+ MB
None
                           order_id  \
0  6b7d50bd145f6fc7f33cebabd7e49d0f   
1  5741ea1f91b5fbab2bd2dc653a5b5099   
2  1ebeea841c590e86a14a0d7a48e7d062   
3  7433cbcc783205509d66a5260da5b574   
4  8428e578bb1cf839ae26a6b7615502b9   

                          review_comment_message product_category_name  labels  
0  O baratheon è esxelente Amo adoro o baratheon         casa_conforto  

In [ ]:
import huggingface_hub
import torch
from google.colab import userdata
from transformers import pipeline

# 로그인
huggingface_hub.login(token=userdata.get('NLP_TOKEN'))

# 모델 및 파이프라인 로드
model_id = "tabularisai/multilingual-sentiment-analysis"
pipe = pipeline("text-classification", model=model_id, device_map="auto")

test_text = "Produto excelente, recomendo!"
test_result = pipe(test_text)[0]
print(f"\n테스트: '{test_text}'")
print(f"결과: {test_result}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0



테스트: 'Produto excelente, recomendo!'
결과: {'label': 'Positive', 'score': 0.7260926961898804}


In [ ]:
from tqdm.auto import tqdm

# 2. 레이블 변환

def parse_label(output):
    """모델 출력 → 0(부정), 1(중립), 2(긍정)"""
    label_map = {
        'Negative': 0, 'negative': 0,
        'Neutral': 1, 'neutral': 1,
        'Positive': 2, 'positive': 2
    }
    label = output.get('label', '') if isinstance(output, dict) else str(output)
    return label_map.get(label.strip(), None)


# 3. 배치 처리 함수
def analyze_sentiment_batch(texts, batch_size=16):
    """
    배치 단위 감성 분석 수행 (Hugging Face Pipeline 활용)

    Args:
        texts: 전처리 완료된 텍스트 리스트
        batch_size: 배치 크기 (GPU 메모리에 따라 조정)

    Returns:
        labels: 0(부정), 1(중립), 2(긍정) 리스트. 유효하지 않은 입력은 None.
    """
    results = [None] * len(texts)

    # 1. 유효성 검사 및 인덱스 추출
    # (주의: 개선된 필터링 로직을 사용합니다.)
    valid_indices = [
        i for i, t in enumerate(texts)
        if pd.notna(t) and str(t).strip() and str(t) != 'nan' and len(str(t).strip()) >= 2
    ]

    if not valid_indices:
        print("⚠️ 유효한 텍스트가 없습니다.")
        return results

    print(f"📊 총 {len(texts):,}개 중 {len(valid_indices):,}개 처리")

    try:
        # 2. 배치 처리 (메인 로직)
        for i in tqdm(range(0, len(valid_indices), batch_size), desc="감성 분석"):
            # 현재 배치의 원본 인덱스
            batch_indices = valid_indices[i:i+batch_size]
            # 현재 배치의 텍스트 내용
            batch_texts = [texts[idx] for idx in batch_indices]

            # 💡 수정: pipe 호출 시 중복 batch_size 인자 제거 (파이프라인이 내부적으로 처리)
            outputs = pipe(batch_texts)

            # 결과 저장
            for idx, out in zip(batch_indices, outputs):
                # outputs는 리스트 형태로 반환되므로, out은 딕셔너리 형태의 결과입니다.
                results[idx] = parse_label(out)

        return results

    except Exception as e:
        print(f"\n❌ 배치 처리 실패: {e}")
        print("→ 개별 처리로 전환...")

        # 3. 개별 처리 폴백 (Fallback)
        for idx in tqdm(valid_indices, desc="개별 처리"):
            try:
                # pipe(texts[idx])는 단일 문장 처리 시 리스트 안에 딕셔너리를 반환합니다.
                # [0]을 사용하여 딕셔너리만 추출하여 parse_label에 전달
                results[idx] = parse_label(pipe(texts[idx])[0])
            except Exception as individual_e:
                # 개별 처리 중 발생하는 오류는 None으로 처리하고 계속 진행
                results[idx] = None
                # print(f"경고: 인덱스 {idx} 처리 실패: {individual_e}") # 디버깅 시 사용

        return results

# 데이터셋 0,1, 2 채우기, 형태 변환:
df_NLP['labels'] = analyze_sentiment_batch(
    df_NLP['review_comment_message'].tolist(),
    batch_size=16
)
df_NLP['labels'].value_counts()

df_NLP.info()

📊 총 33,491개 중 33,461개 처리


감성 분석:   0%|          | 0/2092 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33491 entries, 0 to 33490
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                33491 non-null  object 
 1   review_comment_message  33491 non-null  object 
 2   product_category_name   33491 non-null  object 
 3   labels                  29847 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.0+ MB


In [ ]:
import os
import re
import pandas as pd
import numpy as np
import torch

# ==========================================================
# 0. 설정 및 변수 정의
# ==========================================================

# 💡 개선: label_map_int 정의 추가 (NameError 해결)
# 최종 라벨(final_label)의 값(Neutral, Positive, Negative)에 대응되도록 정의
label_map_int = {"Negative": 0, "Neutral": 1, "Positive": 2}

# 숫자 레이블 → 텍스트 변환 (1.1에서 사용됨)
label_map_reverse = {0: "Negative", 1: "Neutral", 2: "Positive"}

SAVE_PATH = "/content/drive/MyDrive/브라질 이커머스/project_olist_ecommerce/project_dataset/label_quality_check_sample_100.csv"

# 디렉토리 생성
output_dir = os.path.dirname(SAVE_PATH)
os.makedirs(output_dir, exist_ok=True)


# ==========================================================
# 1. 레이블 전처리 및 Unknown 처리 간소화
# ==========================================================

def map_and_handle_na(series, map_dict):
    """
    map을 적용하고, NaN (None, np.nan, float) 값을 "Unknown"으로 처리합니다.
    """
    # .map() 적용
    mapped_series = series.map(map_dict)

    # map에 없는 값(NaN) 및 기존 Unknown/NaN 값 처리
    # np.nan을 "Unknown" 문자열로 변환
    return mapped_series.fillna("Unknown").astype(str)

# 1.1 모델 숫자 레이블(labels) → 텍스트 레이블(labels_str) 변환
# 💡 개선: safe_label 함수 제거 및 map_and_handle_na로 로직 통합
df_NLP["labels_str"] = map_and_handle_na(df_NLP["labels"], label_map_reverse)


# ==========================================================
# 2. 100개 랜덤 샘플 저장 (변동 없음)
# ==========================================================
sample_df = df_NLP.sample(100, random_state=42)[
    ["review_comment_message", "labels_str"]
]
sample_df.to_csv(SAVE_PATH, index=False)


# ==========================================================
# 3. 정책 기반 2nd-pass 단어 정의 (변동 없음)
# ==========================================================

# 확장된 Strong POS
STRONG_POS = ["excelente", "maravilhoso", "ótimo", "perfeito", "adorei", "amei", "sensacional", "muito bom", "recomendo", "super indico", "lindo", "linda", "maravilhosa", "incrível"]
# 확장된 Strong NEG
STRONG_NEG = ["péssimo", "horrível", "porcaria", "muito ruim", "não vale a pena", "decepcionado", "decepção", "arrependi", "péssima", "terrível", "frustrante"]
# 확장된 제품 문제 단어
PRODUCT_PROBLEM = ["quebrado", "estragado", "defeito", "defeituoso", "veio errado", "errado", "produto errado", "faltou", "faltando", "incompleto", "não funciona", "não ligou", "não veio", "partes faltando"]
# 배송 관련
SHIPPING_KEYWORDS = ["entrega", "entregue", "entregaram", "chegou", "envio", "correios", "prazo", "transportadora"]
FAST_SHIPPING = ["antes do prazo", "entrega rápida", "chegou rápido", "chegou antes", "super rápida", "bem rápida"]
# 정책 확정: 미배송/오배송/지연 → Negative
LATE_SHIPPING = ["fora do prazo", "atrasou", "não foi entregue", "não recebi", "atrasada", "atrasado", "extraviado", "perdido", "sumiço"]
# Neutral 패턴
INQUIRY_PATTERNS = ["como faço", "gostaria de saber", "poderiam verificar", "aguardo retorno", "por favor", "pode me ajudar", "quando chega", "tem previsão"]
FACTUAL_PATTERNS = ["como descrito", "recebi corretamente", "não testei", "tudo ok", "ok", "produto está em conformidade", "veio certo", "sem problemas", "produto bom"]


# ==========================================================
# 4. 2nd-pass 규칙 함수 (충돌 제거 + 정책 확정) - 변동 없음
# ==========================================================
def apply_2nd_pass_rules(text: str, model_label: str) -> str:
    t = text.lower().strip()

    # Unknown → Neutral
    if model_label == "Unknown":
        model_label = "Neutral"

    # ========== 우선순위 1: 강한 감정 ==========
    if any(w in t for w in STRONG_POS):
        return "Positive"
    if any(w in t for w in STRONG_NEG):
        return "Negative"

    # ========== 우선순위 2: 제품 문제 ==========
    if any(w in t for w in PRODUCT_PROBLEM):
        return "Negative"

    # ========== 우선순위 3: 배송 문제 (정책: 무조건 Negative) ==========
    if any(w in t for w in LATE_SHIPPING):
        return "Negative"

    # 배송 키워드가 있고, 문제는 아니지만 감정이 없는 경우
    if any(w in t for w in SHIPPING_KEYWORDS):
        # 빠른 배송 → Neutral
        if any(w in t for w in FAST_SHIPPING):
            return "Neutral"
        # 배송 언급만 → Neutral
        return "Neutral"

    # ========== 우선순위 4: 문의/사실 ==========
    if any(w in t for w in INQUIRY_PATTERNS):
        return "Neutral"
    if any(w in t for w in FACTUAL_PATTERNS):
        return "Neutral"

    # ========== 우선순위 5: 혼합 감정(mas) 처리 ==========
    # 강한 POS/NEG가 없고, 제품/배송 문제도 없으면 Neutral
    if " mas " in t or t.startswith("mas ") or "mas," in t:
        return "Neutral"

    # ========== 우선순위 6: 기본 모델 라벨 유지 ==========
    return model_label


# ==========================================================
# 5. 최종 라벨 생성 및 결과 출력
# ==========================================================

# 💡 개선: apply 함수의 입력 변수 간소화. 이미 labels_str이 "Unknown" 문자열로 처리됨
df_NLP["final_label"] = df_NLP.apply(
    lambda row: apply_2nd_pass_rules(
        str(row["review_comment_message"]),
        row["labels_str"]
    ),
    axis=1
)

# 출력 결과 (원래 코드와 동일)
print("▶ Before(모델 라벨)")
print(df_NLP["labels_str"].value_counts(), "\n")

print("▶ After(2nd-pass 보정)")
print(df_NLP["final_label"].value_counts(), "\n")

changed = (df_NLP["labels_str"] != df_NLP["final_label"]).sum()
print(f"총 {changed:,}개 라벨이 보정됨\n")

# df_changed.sample(20, random_state=42) 출력 결과는 여기에 포함하지 않습니다.

# ==========================================================
# 6. 파인튜닝용 데이터셋 준비 및 PyTorch 가중치 계산
# ==========================================================

# 6.1 파인튜닝용 정답 라벨(labels_int) 생성
# 💡 NameError 해결: label_map_int가 0번 섹션에 정의되었으므로 이제 정상 작동
df_NLP["labels_int"] = df_NLP["final_label"].map(label_map_int)

# 7. Fine-Tuning 데이터셋 확정 및 컬럼 이름 정리 (데이터셋 분리)
# 💡 개선: 컬럼 이름 변경 로직은 유지
ft_dataset = df_NLP[["review_comment_message", "labels_int"]].rename(
    columns={"labels_int": "labels"}
)

# 1. 클래스별 데이터 개수 확인 및 리스트로 변환
# 💡 개선: 데이터 순서를 label_map_int의 순서(Neg:0, Neu:1, Pos:2)에 맞게 정렬
label_counts = ft_dataset["labels"].value_counts().sort_index()

counts_list = label_counts.tolist()
total_samples = sum(counts_list)

print(f"총 데이터 개수: {total_samples}")
print(f"클래스별 데이터 개수 (0:Neg, 1:Neu, 2:Pos): {counts_list}\n")

# --- 2. PyTorch 클래스 가중치 계산 및 텐서 생성 ---
# 역빈도(Inverse Frequency) 기반 가중치 공식 사용

# 각 클래스 개수의 역수를 구합니다.
reciprocal_counts = [total_samples / count for count in counts_list]

# PyTorch 텐서 생성
class_weights = torch.tensor(reciprocal_counts, dtype=torch.float)

# 결과 출력
print("✅ PyTorch 클래스 가중치 텐서")
print(f"순서(0: Negative, 1: Neutral, 2: Positive): {class_weights}")

▶ Before(모델 라벨)
labels_str
Positive    13704
Negative     9061
Neutral      7082
Unknown      3644
Name: count, dtype: int64 

▶ After(2nd-pass 보정)
final_label
Neutral     14301
Positive    12883
Negative     6307
Name: count, dtype: int64 

총 13,076개 라벨이 보정됨

총 데이터 개수: 33491
클래스별 데이터 개수 (0:Neg, 1:Neu, 2:Pos): [6307, 14301, 12883]

✅ PyTorch 클래스 가중치 텐서
순서(0: Negative, 1: Neutral, 2: Positive): tensor([5.3101, 2.3419, 2.5996])


In [ ]:
from sklearn.model_selection import train_test_split

# 1. 데이터셋 정의
X = ft_dataset['review_comment_message']
y = ft_dataset['labels']
RANDOM_SEED = 42 # 재현성을 위한 시드 고정

# 2. 1차 분리: 훈련 셋 (80%) vs 임시 셋 (20%)
# test_size=0.2 (임시 셋 비율)
train_X, temp_X, train_y, temp_y = train_test_split(
    X, y,
    test_size=0.2,
    random_state=RANDOM_SEED,
    stratify=y # 레이블 비율 유지
)

# 3. 2차 분리: 임시 셋 (20%) → 평가 셋 (10%) vs 테스트 셋 (10%)
# temp_X의 50%를 평가 셋(10%), 나머지 50%를 테스트 셋(10%)으로 사용
# test_size=0.5 (temp_X의 절반)
# stratify=temp_y 를 사용하여 남은 레이블 비율 유지
val_X, test_X, val_y, test_y = train_test_split(
    temp_X, temp_y,
    test_size=0.5,
    random_state=RANDOM_SEED,
    stratify=temp_y
)

total_count = len(X)
train_count = len(train_X)
val_count = len(val_X)
test_count = len(test_X)

print("--- 📊 데이터셋 크기 확인 ---")
print(f" 총 데이터 개수: {total_count:,}개")
print(f" 훈련 셋 (80%): {train_count:,}개 ({(train_count / total_count) * 100:.2f}%)")
print(f" 평가 셋 (10%): {val_count:,}개 ({(val_count / total_count) * 100:.2f}%)")
print(f" 테스트 셋 (10%): {test_count:,}개 ({(test_count / total_count) * 100:.2f}%)")
print("-" * 30)

# ==========================================================
# 2. 클래스 비율 (Stratification) 검증
# ==========================================================
# 클래스 순서: 0: Negative, 1: Neutral, 2: Positive
# 원본 데이터의 Negative 비율 (6307 / 33491 ≈ 18.83%)

def get_class_ratios(labels):
    """주어진 레이블 셋의 클래스별 비율을 계산합니다."""
    # normalize=True 옵션으로 각 레이블의 비율을 계산
    return labels.value_counts(normalize=True).sort_index() * 100

print("\n--- ⚖️ 클래스 비율 (Negative) 검증 ---")

# 원본 레이블 비율
original_ratios = get_class_ratios(y)
neg_ratio_original = original_ratios.loc[0]
print(f"✅ 원본 Negative 비율: {neg_ratio_original:.2f}%")

# 훈련 셋 레이블 비율
train_ratios = get_class_ratios(train_y)
neg_ratio_train = train_ratios.loc[0]
print(f"1️⃣ 훈련 셋 Negative 비율: {neg_ratio_train:.2f}%")

# 평가 셋 레이블 비율
val_ratios = get_class_ratios(val_y)
neg_ratio_val = val_ratios.loc[0]
print(f"2️⃣ 평가 셋 Negative 비율: {neg_ratio_val:.2f}%")

# 테스트 셋 레이블 비율
test_ratios = get_class_ratios(test_y)
neg_ratio_test = test_ratios.loc[0]
print(f"3️⃣ 테스트 셋 Negative 비율: {neg_ratio_test:.2f}%")
print("-" * 30)

# ==========================================================
# 3. 전체 클래스 분포 비교 (선택 사항)
# ==========================================================
print("\n--- 전체 클래스 분포 (0:Neg, 1:Neu, 2:Pos) ---")
print("원본\n", original_ratios.round(2))
print("훈련 셋\n", train_ratios.round(2))
print("평가 셋\n", val_ratios.round(2))
print("테스트 셋\n", test_ratios.round(2))

--- 📊 데이터셋 크기 확인 ---
 총 데이터 개수: 33,491개
 훈련 셋 (80%): 26,792개 (80.00%)
 평가 셋 (10%): 3,349개 (10.00%)
 테스트 셋 (10%): 3,350개 (10.00%)
------------------------------

--- ⚖️ 클래스 비율 (Negative) 검증 ---
✅ 원본 Negative 비율: 18.83%
1️⃣ 훈련 셋 Negative 비율: 18.83%
2️⃣ 평가 셋 Negative 비율: 18.84%
3️⃣ 테스트 셋 Negative 비율: 18.84%
------------------------------

--- 전체 클래스 분포 (0:Neg, 1:Neu, 2:Pos) ---
원본
 labels
0    18.83
1    42.70
2    38.47
Name: proportion, dtype: float64
훈련 셋
 labels
0    18.83
1    42.70
2    38.47
Name: proportion, dtype: float64
평가 셋
 labels
0    18.84
1    42.70
2    38.46
Name: proportion, dtype: float64
테스트 셋
 labels
0    18.84
1    42.69
2    38.48
Name: proportion, dtype: float64


In [ ]:
from transformers import AutoTokenizer

# 토크나이저 로드 (모델과 동일한 토크나이저 사용)
tokenizer = AutoTokenizer.from_pretrained("tabularisai/multilingual-sentiment-analysis")

print("Tokenizer loaded successfully.")

# train_X, val_X, test_X, train_y, val_y, test_y Series를 DataFrame으로 변환
train_df = pd.DataFrame({'review_comment_message': train_X, 'labels': train_y})
val_df = pd.DataFrame({'review_comment_message': val_X, 'labels': val_y})
test_df = pd.DataFrame({'review_comment_message': test_X, 'labels': test_y})

# 인덱스 초기화
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print("DataFrames train_df, val_df, test_df created successfully.")
print(f"train_df head:\n{train_df.head()}")

from datasets import Dataset, DatasetDict

# 1. 데이터프레임을 Hugging Face Dataset으로 변환

train_hf_dataset = Dataset.from_pandas(train_df)
val_hf_dataset = Dataset.from_pandas(val_df)
test_hf_dataset = Dataset.from_pandas(test_df)

# DatasetDict으로 묶어서 관리
raw_datasets = DatasetDict({
    'train': train_hf_dataset,
    'validation': val_hf_dataset,
    'test': test_hf_dataset
})

# 2. 토크나이징 함수 정의
def tokenize_function(examples):
    return tokenizer(
        examples['review_comment_message'],
        truncation=True,
        padding='max_length',
        max_length=256 # 모델 입력 길이에 맞춰 조정
    )

# 3. 모든 데이터셋에 토크나이징 적용
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=['review_comment_message'] # '__index_level_0__' 제거
)

# 결과 확인
print("--- 토크나이징 완료된 Dataset 구조 ---")
print(tokenized_datasets)
print(f"훈련 데이터셋 첫 번째 샘플:\n{tokenized_datasets['train'][0]}")

Tokenizer loaded successfully.
DataFrames train_df, val_df, test_df created successfully.
train_df head:
                              review_comment_message  labels
0  bom produto,estou sastifeito com o produto eu ...       2
1  Recebi num prazo menor que o estimado, produto...       1
2                   O relógio tem uma boa qualidade.       2
3  Perfume original, muito bom. Porém houve muita...       2
4                   Atingiu minhas expectativas 👏🏽👏🏽       2


Map:   0%|          | 0/26792 [00:00<?, ? examples/s]

Map:   0%|          | 0/3349 [00:00<?, ? examples/s]

Map:   0%|          | 0/3350 [00:00<?, ? examples/s]

--- 토크나이징 완료된 Dataset 구조 ---
DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 26792
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3349
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3350
    })
})
훈련 데이터셋 첫 번째 샘플:
{'labels': 2, 'input_ids': [101, 32965, 67727, 117, 18278, 10138, 10148, 11964, 14601, 12390, 10212, 183, 67727, 14444, 11639, 61643, 10605, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 1. 모델 로드 (3개 클래스)
model = AutoModelForSequenceClassification.from_pretrained(
    "tabularisai/multilingual-sentiment-analysis",
    num_labels=3, # Negative, Neutral, Positive
    ignore_mismatched_sizes=True # 크기 불일치 무시하고 분류 헤드 재초기화
)

# 2. 데이터 콜레이터 정의
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. 평가 지표 정의 (정확도, 정밀도, 재현율, F1-점수)
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted'
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

print("모델, 데이터 콜레이터, 평가 지표 설정 완료.")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at tabularisai/multilingual-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


모델, 데이터 콜레이터, 평가 지표 설정 완료.


In [ ]:
from transformers import Trainer

# TrainingArguments는 이미 2CPUUrec8As_ 셀에 정의되어 있습니다.
# args = TrainingArguments(
#     output_dir='/content/drive/MyDrive/project_olist_ecommerce/checkpoints',
#     eval_strategy="steps",
#     save_strategy="steps",
#     eval_steps=500,
#     save_steps=500,
#     logging_steps=100,
#     save_total_limit=2,
#     load_best_model_at_end=True
# )

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("모델 학습을 시작합니다...")

/tmp/ipython-input-1828183847.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


모델 학습을 시작합니다...


In [ ]:
import os
from transformers.trainer_utils import get_last_checkpoint # 추가: get_last_checkpoint 함수 임포트

# 1. 체크포인트가 저장되는 경로
# TrainingArguments에서 설정한 기본 output_dir과 동일하게 설정합니다.
output_dir = '/content/drive/MyDrive/project_olist_ecommerce/checkpoints'

# 2. 유효한 체크포인트 경로를 찾습니다.
last_checkpoint_path = get_last_checkpoint(output_dir)

# 3. 학습 시작
if last_checkpoint_path is not None:
    print(f"✅ 유효한 체크포인트를 발견했습니다! '{last_checkpoint_path}'부터 학습을 재개합니다.")
    trainer.train(resume_from_checkpoint=last_checkpoint_path)
else:
    print("⚠️ 유효한 체크포인트를 발견할 수 없습니다.")
    print("🚀 처음부터 학습을 새로 시작합니다.")
    trainer.train()

⚠️ 유효한 체크포인트를 발견할 수 없습니다.
🚀 처음부터 학습을 새로 시작합니다.


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: